In [ ]:
# Importing library modules

from google.colab import files                               # To download files
import numpy as np                                           # For array operation
import matplotlib.pyplot as plt                              # For plotting
import glob                                                  # To dynamically load data
import cv2                                                   # Image related operation
from google.colab.patches import cv2                         # For displaying image
from tensorflow.keras.preprocessing import image             # Image processing
from tensorflow.keras.applications.densenet import DenseNet121     # To import convolution base
from tensorflow.keras.applications.densenet import preprocess_input # To preprocess images according to the model
from keras.models import Model, Sequential                   # To import model skeleton
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D # To import layer and related methods, classes
import os                                                     # To find the path
import seaborn as sns                                         # For editing plots

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importing data from MyDrive

print(os.listdir("/content/drive/MyDrive/FINAL(ORG)"))

SIZE = 128

Timages = []
Tlabels = []
for directory_path in glob.glob("/content/drive/MyDrive/FINAL(ORG)/*"):
    label = directory_path.split("/")[-1]
    print(label)
    print(directory_path)
    for img_path in glob.glob(os.path.join(directory_path,"*.*")):
      print(img_path)
      img = cv2.imread(img_path, cv2.IMREAD_COLOR)
      img = cv2.resize(img, (SIZE, SIZE))
      img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
      img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
      Timages.append(img)
      Tlabels.append(label)

Timages = np.array(Timages)
Tlabels = np.array(Tlabels)

In [ ]:
# Splitting data between training and testing sets

from sklearn.model_selection import train_test_split

# If integer encoding required
#x_train, x_val, y_train, y_val = train_test_split(Timages, Tlabels_encoded, test_size=0.33, random_state=42)

# If one hot encoding required
x_train, x_test, y_train, y_test = train_test_split(Timages, Tlabels, test_size=0.20, random_state=42)
#x_train, x_val, y_train, y_val = train_test_split(x_t, y_t, test_size=0.33, random_state=40)

In [ ]:
#Encode labels from text to integers.

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)

#le.fit(y_val)
#y_val = le.transform(y_val)

In [ ]:
# Perform one hot encoding

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
#y_val = to_categorical(y_val)

In [ ]:
# Importing pre-trainned cnn 'MobileNetV2'

model = DenseNet121(input_shape = (128, 128, 3), weights='imagenet', include_top=False)
print(len(model.layers))

427


In [ ]:
#Add layers for feature extraction in deep learning

#activation = 'sigmoid'
#'sigmoid'
x = model.output
feature_layer = Flatten()(x)

# Make a new model combining both feature extractor and x
cnn_model = Model(inputs=model.input, outputs=feature_layer)

In [ ]:
# Freeznig layers and compiling the model

for layers in (cnn_model.layers)[:427]:
    print(layers)
    layers.trainable = False

#cnn_model.compile(optimizer='Adam',loss = 'binary_crossentropy', metrics = ['accuracy'])
print(cnn_model.summary())

In [ ]:
# Extract the features using the deep learning architecture

x_train_feature = cnn_model.predict(x_train) #This is out X input to Machine Classifier

8/8 [==============================] - 31s 3s/step


In [ ]:
# Immport machine learning classifier
#from sklearn.neighbors import KNeighborsClassifier
#MC_model = KNeighborsClassifier()
#from sklearn.ensemble import RandomForestClassifier
#MC_model = RandomForestClassifier(n_estimators = 50, random_state = 42)
#from sklearn.ensemble import AdaBoostClassifier
#MC_model = AdaBoostClassifier(n_estimators=100)
#from sklearn.ensemble import GradientBoostingClassifier
#MC_model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
from sklearn.linear_model import LogisticRegression
MC_model = LogisticRegression(random_state=1)


In [ ]:
# Inverse transform training labels

y_train = np.argmax(y_train,1)

In [ ]:
# Train the model on training data

MC_model.fit(x_train_feature, y_train)                # x - Image, y - Label

X_test_feature = cnn_model.predict(x_test)
prediction_MC_Unravel = MC_model.predict(X_test_feature)
prediction_MC = prediction_MC_Unravel
print(prediction_MC)
prediction_MC = le.inverse_transform(prediction_MC)
print(len(x_test))
print(prediction_MC)

In [ ]:
# Draw learning curves with cross-validation

import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from yellowbrick.model_selection import LearningCurve

# Input the data
X = x_train_feature
y_train_decoded = y_train
y = y_train_decoded

# Create the learning curve visualizer
cv = 4
#cv = StratifiedKFold(n_splits=12)
sizes = np.linspace(0.1, 1.0, 20)

# Instantiate the classification model and visualizer
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(4)
    ax.spines[axis].set_color('brown')

plt.xticks(fontsize=17,weight='bold')
plt.yticks(fontsize=17,weight='bold')

#ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

visualizer = LearningCurve(MC_model, cv=cv, scoring='accuracy', train_sizes=sizes, ax=ax, kws = {'linewidths': [100,100]})
visualizer.fit(X, y)        # Fit the data to the visualizer

visualizer.ax.set_xlabel('Training instances',fontsize=25,color='brown',weight='bold')
visualizer.ax.set_ylabel('Score',fontsize=25,color='brown',weight='bold')
visualizer.ax.set_title('DenseNet121+LR',fontsize=25,color='brown',weight='bold')
plt.xticks(fontsize=17,weight='bold')
plt.yticks(fontsize=17,weight='bold')
plt.legend(loc=(0.56,0.03), frameon=True, prop={'size':10,'weight':'bold'})

plt.savefig("DenseNet121+LR.pdf", bbox_inches='tight')
files.download("DenseNet121+LR.pdf")

visualizer.show()           # Finalize and render the figure

In [ ]:
#Print overall accuracy

from sklearn import metrics
from sklearn.metrics import confusion_matrix
print(len(y_test))
print(len(prediction_MC))
print ("Accuracy = ", metrics.accuracy_score(y_test, prediction_MC))

#Confusion Matrix - verify accuracy of each class
cm = confusion_matrix(y_test, prediction_MC)

accuracy=metrics.accuracy_score(y_test,prediction_MC)
precision_positive=metrics.precision_score(y_test,prediction_MC,pos_label='Smoke_A')
print('precision=',precision_positive)
recall_sensitivity=metrics.recall_score(y_test,prediction_MC,pos_label='Smoke_A')
print('Recall=',recall_sensitivity)
f1_positive=metrics.f1_score(y_test,prediction_MC,pos_label='Smoke_A')
print('F1-score=',f1_positive)
hamming_loss=metrics.hamming_loss(y_test, prediction_MC)
print('hamming loss=',hamming_loss)

#jaccard_score=metrics.jaccard_score(y_test_metric, prediction_MC_metric)
#print('jaccard score=',jaccard_score)
x_test_feat=cnn_model.predict(x_test)
y_test_prob=MC_model.predict_proba(x_test_feat)
cross_entropy_loss=metrics.log_loss(y_test,y_test_prob)
print('Cross entropy Loss=',cross_entropy_loss)

#print(cm)
sns.heatmap(cm, annot=True)

In [ ]:

from sklearn.metrics import roc_curve
from sklearn.metrics import auc

X_test_feature = cnn_model.predict(x_test)
y_scores = MC_model.predict_proba(X_test_feature)
#print(y_scores[0])
#print(y_test)
#print(y_scores[1])
#print(y_test)

le.fit(y_test)
y_test_trans = le.transform(y_test)
#print(y_test)

fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)

for axis in ['top', 'bottom', 'left', 'right']:
  ax.spines[axis].set_linewidth(5)
  ax.spines[axis].set_color('black')

fpr, tpr, threshold = roc_curve(y_test_trans, y_scores[:,1])
roc_auc = auc(fpr, tpr)

plt.xticks(fontsize=20,weight='bold')
plt.yticks(fontsize=20,weight='bold')
plt.plot([0, 1], [0, 1], 'k--',linewidth=5)
plt.plot(fpr, tpr, color='#008929', label='AUC = {:.3f}'.format(roc_auc),linewidth=5)
plt.legend(loc='best', frameon=True, prop={'size':15,'weight':'bold'})
plt.xlim([-0.05, 1])
plt.ylim([0, 1.05])
plt.xlabel('False positive rate',fontsize=22,color='brown',weight='bold')
plt.ylabel('True positive rate',fontsize=22,color='brown',weight='bold')
plt.title('DenseNet121+LR',fontsize=25,color='brown',weight='bold')
plt.grid(False)

plt.savefig("ROC_DenseNet121+LR.pdf", bbox_inches='tight')
files.download("ROC_DenseNet121+LR.pdf")

plt.show()